In [1]:
import os

root_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
os.chdir(root_dir)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import functions
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.6)



# Kyte & Doolittle index of hydrophobicity
# J. Mol. Biol. 157:105-132(1982).
kd = {"A": 1.8, "R": -4.5, "N": -3.5, "D": -3.5, "C": 2.5,
      "Q": -3.5, "E": -3.5, "G": -0.4, "H": -3.2, "I": 4.5,
      "L": 3.8, "K": -3.9, "M": 1.9, "F": 2.8, "P": -1.6,
      "S": -0.8, "T": -0.7, "W": -0.9, "Y": -1.3, "V": 4.2}

#Sort kd
kd = {k: v for k, v in sorted(kd.items(), key=lambda item: item[1])}
aa = [k for k,v in kd.items()]

%load_ext autoreload
%autoreload 2

In [3]:
train = pd.read_pickle('results/signalp5_train_all.pkl.gz')

In [4]:
df1 = pd.read_csv('data/uniprot_new/uniprot-keyword Signal+[KW-0732] +AND+reviewed yes+AND+taxonomy %--.tab.gz', sep='\t')
df1.head(2)

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence,Signal peptide,Protein existence,Taxonomic lineage (all)
0,A4IGY6,S39AE_XENTR,reviewed,Metal cation symporter ZIP14 (Solute carrier f...,slc39a14,Xenopus tropicalis (Western clawed frog) (Silu...,462,MPLLLLSALLPFSLMAGPTPSTGKELSAASFLQDILQRYGENESLS...,"SIGNAL 1..16; /evidence=""ECO:0000255""",Evidence at transcript level,"cellular organisms, Eukaryota, Opisthokonta, M..."
1,Q08535,SECR_MOUSE,reviewed,Secretin,Sct,Mus musculus (Mouse),133,MEPPLPTPMLLLLLLLLSSSAALPAPPRTPRHSDGMFTSELSRLQD...,"SIGNAL 1..22; /evidence=""ECO:0000255""",Evidence at transcript level,"cellular organisms, Eukaryota, Opisthokonta, M..."


#### New SP

In [5]:
df1_ = df1[df1.Entry.apply(lambda x: x not in train.Entry.values)]
## Take experimental ones
df1 = df1_[df1_['Signal peptide'].apply(lambda x: '000269' in str(x))].copy()

In [6]:
df1['Label'] = 1

### New NO_SP

In [7]:
df2 = pd.read_csv('data/uniprot_new/uniprot-taxonomy Eukaryota+[2759] +AND+reviewed yes.tab.gz', sep='\t')
## Take experimental ones
df2 = df2[df2['Protein existence'].apply(lambda x: 'protein level' in str(x).lower())].copy()
# Remove training
df2 = df2[df2.Entry.apply(lambda x: x not in train.Entry.values)]

In [8]:
df2 = df2[df2['Signal peptide'].isna()]

In [9]:
df2['Label'] = 0

#### Combine 

In [10]:
new_df = pd.concat([df1, df2]).reset_index(drop=True)

In [11]:
new_df.to_pickle('results/new_uniprot.pkl.gz')

In [12]:
new_df['Protein existence'].value_counts()

Evidence at protein level    53026
Name: Protein existence, dtype: int64

In [13]:
new_df.Label.value_counts()

0    52735
1      291
Name: Label, dtype: int64